# Часть 1 Бустинг

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [ ]:
import pandas as pd

df = pd.read_csv("ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1: Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['salary', 'salary_in_usd'], axis=1)
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Возможен лик данных из-за признака salary, так как это просто таргет переменная в другой валюте.

## Задание 2: Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np

categorical_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size']

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
ohed_train = ohe.fit_transform(X_train[categorical_features])
ohed_val = ohe.transform(X_val[categorical_features])
ohed_test = ohe.transform(X_test[categorical_features])

X_train = pd.concat([X_train, ohed_train], axis=1).drop(columns=categorical_features)
X_val = pd.concat([X_val, ohed_val], axis=1).drop(columns=categorical_features)
X_test = pd.concat([X_test, ohed_test], axis=1).drop(columns=categorical_features)

linreg = LinearRegression().fit(X_train, y_train)
pred = linreg.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, pred)))

MAPE:  21190512.49481572
RMSE:  9201339294801.95


Очень высокие показатели функций ошибки

## Задание 3: XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
from xgboost.sklearn import XGBRegressor

min_mape = float('inf')
params = {
    'max_depth' : None,
    'learning_rate' : None,
    'n_estimators' : None,
    'gamma' : None
}

for depth in range(1, 6, 2):
  for lr in [0.01, 0.1]:
    for n in range(900, 1200, 100):
      for gamma in [0.001, 0.01]:
        xgb = XGBRegressor(max_depth=depth, learning_rate=lr, n_estimators=n, gamma=gamma).fit(X_train, y_train)
        pred = xgb.predict(X_val)

        mape = mean_absolute_percentage_error(y_val, pred)
        if mape < min_mape:
          min_mape = mape
          params = {'max_depth' : depth, 'learning_rate' : lr, 'n_estimators' : n, 'gamma' : gamma}

params

{'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 1100, 'gamma': 0.001}

In [ ]:
xgboost = XGBRegressor()
xgboost.set_params(**params)
xgboost.fit(X_train, y_train)

pred = xgboost.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, pred)))

MAPE:  0.2862052501946971
RMSE:  44876.17328450639


Показатели стали значительно лучше, но всё ещё не идеальны. Скорость хорошая, модель обучилась и предсказала за 3 секунды. Подбор гиперпараметров занял 1 мин. 42 сек.

## Задание 4: CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

min_mape = float('inf')
params = {
    'max_depth' : None,
    'learning_rate' : None,
    'iterations': None
}

for depth in range(1, 6):
  for lr in [0.01, 0.1]:
    for n in range(300, 800, 100):
      cat = CatBoostRegressor(verbose=False, max_depth=depth, learning_rate=lr, iterations=n).fit(X_train, y_train)
      pred = cat.predict(X_val)

      mape = mean_absolute_percentage_error(y_val, pred)
      if mape < min_mape:
        min_mape = mape
        params = {'max_depth' : depth, 'learning_rate' : lr, 'iterations' : n}

params

{'max_depth': 5, 'learning_rate': 0.1, 'iterations': 700}

In [ ]:
catboost = CatBoostRegressor(verbose=False)
catboost.set_params(**params)
catboost.fit(X_train, y_train)

pred = catboost.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, pred)))

MAPE:  0.306358341244474
RMSE:  44920.44763924746


Показатели MAPE и RMSE практически такие же, даже немного больше, чем на XGBoost. Скорость значительно быстрее, гиперпараметры подобрались за 30 секунд, а сама модель работала 1 секунду.

Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [ ]:
from catboost import Pool

X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.2, random_state=42)
X1_val, X1_test, y1_val, y1_test = train_test_split(X1_test, y1_test, test_size=0.5, random_state=42)

train_pool = Pool(data=X1_train, label=y1_train, cat_features=categorical_features)
test_pool = Pool(data=X1_test, cat_features=categorical_features)
val_pool = Pool(data=X1_val, cat_features=categorical_features)

In [ ]:
min_mape = float('inf')
params = {
    'max_depth' : None,
    'learning_rate' : None,
    'iterations': None
}

for depth in range(1, 6):
  for lr in [0.01, 0.1]:
    for n in range(300, 800, 100):
      cat = CatBoostRegressor(verbose=False, max_depth=depth, learning_rate=lr, iterations=n).fit(train_pool)
      pred = cat.predict(val_pool)

      mape = mean_absolute_percentage_error(y1_val, pred)
      if mape < min_mape:
        min_mape = mape
        params = {'max_depth' : depth, 'learning_rate' : lr, 'iterations' : n}

params

{'max_depth': 4, 'learning_rate': 0.1, 'iterations': 700}

In [ ]:
catboost = CatBoostRegressor(verbose=False)
catboost.set_params(**params)
catboost.fit(train_pool)

pred = catboost.predict(test_pool)

print('MAPE: ', mean_absolute_percentage_error(y1_test, pred))
print('RMSE: ', np.sqrt(mean_squared_error(y1_test, pred)))

MAPE:  0.34972452833553735
RMSE:  50341.301527084135


**Ответ:** Качество сильно не поменялось, даже немного ухудшилось. Зато скорость немного уменьшилась.

## Задание 5: LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [ ]:
from lightgbm import LGBMRegressor


params = {
    'max_depth' : None,
    'learning_rate' : None,
    'n_estimators' : None,
}

min_mape = float('inf')

for depth in range(1, 6):
  for lr in [0.01, 0.1]:
    for n in range(800, 1200, 100):
      lgbm = LGBMRegressor(verbose=-1, max_depth=depth, learning_rate=lr, n_estimators=n).fit(X_train, y_train)
      pred = lgbm.predict(X_val)

      mape = mean_absolute_percentage_error(y_val, pred)
      if mape < min_mape:
        min_mape = mape
        params = {'max_depth' : depth, 'learning_rate' : lr, 'n_estimators' : n}

params

{'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 1100}

In [ ]:
lightgbm = LGBMRegressor(verbose=-1)
lightgbm.set_params(**params)
lightgbm.fit(X_train, y_train)

pred = lightgbm.predict(X_test)

print('MAPE: ', mean_absolute_percentage_error(y_test, pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, pred)))

MAPE:  0.3430057004967812
RMSE:  46987.21675678158


Показатели получились соизмеримыми с CatBoost, однако работает модель в разы быстрее, чем предыдущие.

## Задание 6: Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

In [ ]:
boosts = {'XGBoost': [0.29, 44876.17, 102, 3], 'CatBoost': [0.31, 44920.45, 28, 1], 'Cat_Pools': [0.35, 50341.3, 65, 4], 'LightGBM': [0.34, 46987.22, 12, 0]}

pd.DataFrame(boosts, index=['MAPE', 'RMSE', 'Гиперпараметры', 'Модель'])

,XGBoost,CatBoost,Cat_Pools,LightGBM
MAPE,0.29,0.31,0.35,0.34
RMSE,44876.17,44920.45,50341.30,46987.22
Гиперпараметры,102.00,28.00,65.00,12.00
Модель,3.00,1.00,4.00,0.00


**Ответ:** Для наиболее удобного сравнения я вывела таблицу значений MAPE, RMSE, длительности поиска гиперпараметров и работы модели (в секундах).
Все модели показали в среднем одинаковые результаты, около 30% для MAPE и RMSE в среднем 45000 (за исключением CatBoost с использованием пулов, у него RMSE вышел больше). Это достаточно хорошие результаты, но конечно далёкие от идеала. Помимо подбора гиперпараметров в целом все модели работали довольно быстро, но LightGBM была в разы быстрее остальных моделей. CatBoost с использованием пулов у меня вышел дольше, чем обычный CatBoost, что довольно странно. Гиперпараметры получились везде сильно похожими, learning rate у всех моделей был 0.1, глубина 4-5, n_estimators равное 1100, а оптимальное число итераций для CatBoost - 700.

# Часть 2 Кластеризация

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [ ]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1: Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [ ]:
ratings = ratings.T
ratings.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
user,0.000000,1.000000,2.000000,3.0,4.000000,5.000000,6.0,7.0,8.000000,9.000000,...,4990.000000,4991.0,4992.000000,4993.000000,4994.000000,4995.000000,4996.0,4997.000000,4998.0,4999.000000
the beatles,NaN,NaN,NaN,NaN,0.043529,NaN,NaN,NaN,0.093398,0.017621,...,NaN,NaN,0.121169,0.038168,0.007939,0.017884,NaN,0.076923,NaN,NaN
radiohead,0.020417,0.184962,NaN,NaN,0.086281,0.006322,NaN,NaN,NaN,0.019156,...,0.017735,NaN,NaN,NaN,0.011187,NaN,NaN,NaN,NaN,NaN
deathcab for cutie,NaN,0.024561,0.028635,NaN,0.034590,NaN,NaN,NaN,NaN,0.013349,...,0.121344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027893
coldplay,NaN,NaN,NaN,NaN,0.016712,NaN,NaN,NaN,NaN,NaN,...,0.217175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Выкиньте строку под названием `user`.

In [ ]:
ratings = ratings.drop(['user'])

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [ ]:
ratings = ratings.fillna(0)
ratings.sample()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
streetlight manifesto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.05473,0.0,0.0,0.0,0.0


## Задание 2: Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=5, random_state=42).fit(ratings)

labels = km.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [ ]:
sizes = pd.Series(labels).value_counts()
sizes

2    995
0      2
3      1
4      1
1      1
Name: count, dtype: int64

**Ответ:** Практически все данные попали в один кластер, тогда как остальные кластеры состоят из 1-2 элементов. Такой результат может указать на недостаточную обработку данных (отсутствие нормализации).

## Задание 3: Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [ ]:
smalls = sizes[sizes == 1].index
smalls_artists = ratings.index[labels == smalls[0]].tolist()

for cluster in smalls[1:]:
    smalls_artists.extend(ratings.index[labels == cluster].tolist())

smalls_artists

['the beatles', 'coldplay', '보아']

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [ ]:
percentage_users = np.mean(ratings > 0, axis=1)
average_listens = np.mean(ratings, axis=1)

pd.DataFrame({'Percantage of users': percentage_users, 'Average Listens': average_listens}).head()

,Percantage of users,Average Listens
the beatles,0.3342,0.018369
radiohead,0.2778,0.011851
deathcab for cutie,0.1862,0.006543
coldplay,0.1682,0.006030
modest mouse,0.1628,0.005876


**Ответ:** Потому что у the beatles самая большая доля прослушиваний, как и доля пользователей, которые их слушают, причём с достаточно значительным разрывом.

## Задание 4: Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [ ]:
from sklearn.preprocessing import normalize

df = ratings
ratings = normalize(ratings)

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [ ]:
km = KMeans(n_clusters=5, random_state=42).fit(ratings)

labels = km.labels_
pd.Series(labels).value_counts()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


2    405
0    237
1    152
3    139
4     67
Name: count, dtype: int64

**Ответ:** Данное разделение является намного более репрезентатиивным, размеры кластеров пропорциональны друг другу.

## Задание 5: Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_
top = {}
ratings = pd.DataFrame(ratings)

for i in range(5):
    centroid = centroids[i]
    dists = []
    for j in range(len(ratings)):
        artist_vector = ratings.iloc[j]
        dist = cosine(centroid, artist_vector)
        dists.append((dist, ratings.index[j]))
    dists.sort()
    top_10 = [artist for _, artist in dists[:10]]
    top[i] = top_10


top10 = {}
for i, artists in top.items():
    arts = []
    for artist in artists:
        arts.append(df.index[artist])
    top10.update({f'Cluster {i + 1}' : arts})

pd.DataFrame(top10)

,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5
0,radiohead,kelly clarkson,the beatles,fall out boy,nas
1,the arcade fire,rihanna & jay-z,the rolling stones,saosin,jay-z
2,broken social scene,maroon5,led zeppelin.,brand new,a tribe called quest
3,animal collective,the pussycat dolls,pink fluid,taking back sunday,kanye west
4,belle and sebastian,john mayer,acdc,blink-182,the roots featuring d'angelo
5,sufjan stevens,alicia keys,metallica,anberlin,lupe the gorilla
6,the shins,lady gaga,radiohead,the used,gangstarr
7,of montreal,beyoncé,red hot clili peppers,cartel,murs and 9th wonder
8,the pixies,coldplay,the clash,chiodos,little brother
9,spoon,nelly furtado,queen,new found glory,mos def


**Ответ:** Данное разделение на кластеры является намного более логичным, и даже можно заметить определённые схожести в стиле исполнителей одного кластера.